In [27]:
import spacy
#!pip install PyPDF2

import PyPDF2

from sklearn.metrics.pairwise import cosine_similarity

# Load the spaCy English model
nlp = spacy.load('en_core_web_sm')

# Load the job posting and the CV
job_posting = open('/content/CSharpJobDesc.pdf', 'rb')
job_pdf_reader = PyPDF2.PdfReader(job_posting)
job_text = ''
for i in range(len(job_pdf_reader.pages)):
    job_text += str(job_pdf_reader.pages[i].extract_text)
job_posting.close()

cv = open('/content/IbrahimAliyev.pdf', 'rb')
cv_pdf_reader = PyPDF2.PdfReader(cv)
cv_text = ''
for i in range(len(cv_pdf_reader.pages)):
    cv_text += str(cv_pdf_reader.pages[i].extract_text())
cv.close()

# Define a function to clean the text
def clean_text(text):
    doc = nlp(text)
    cleaned_text = []
    for token in doc:
        if not token.is_stop and not token.is_punct and not token.is_space:
            cleaned_text.append(token.lemma_.lower())
    return ' '.join(cleaned_text)

# Clean the CV
cleaned_cv = clean_text(cv_text)

# Extract information from the job posting
job_title = nlp(job_text).ents[0].text
job_skills = [ent.text for ent in nlp(job_text).ents if ent.label_ == 'Skills']

# Extract information from the CV
cv_skills = [ent.text for ent in nlp(cleaned_cv).ents if ent.label_ == 'SKILLS']

# Convert extracted information to vectors
job_skills_vector = nlp(' '.join(job_skills)).vector
cv_skills_vector = nlp(' '.join(cv_skills)).vector

# Calculate the similarity score
similarity_score = cosine_similarity([job_skills_vector], [cv_skills_vector])[0][0]
print(similarity_score)

# Rank the CVs based on their similarity score
if similarity_score > 0.7:
    print("CV is a good match")
else:
    print("CV is not a good match")


ValueError: ignored